Example video used here (Credit to National Geographic): https://www.youtube.com/watch?v=oBb9vEWBYZo

# Imports & Installations

In [1]:
!pip install opencv-python-headless
!pip install torch torchvision torchaudio
!pip install git+https://github.com/ultralytics/yolov5.git

  Cloning https://github.com/ultralytics/yolov5.git to /tmp/pip-req-build-0xng5483
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/yolov5.git /tmp/pip-req-build-0xng5483
  Resolved https://github.com/ultralytics/yolov5.git to commit 19ce90290c6bd013b69dbe037e8ef1bc3d3c32d0
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
import cv2
import os
import torch
from pathlib import Path
from PIL import Image
import numpy as np

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Video Partitioning

In [5]:
def split_video_to_frames(video_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    count = 0

    i = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if count % fps == 0:
            seconds = count // fps
            frame_filename = os.path.join(output_folder, f'{seconds}.jpg')
            cv2.imwrite(frame_filename, frame)
        count += 1
        print("Frame count:", i)
        i += 1

    cap.release()

In [ ]:
video_path = 'path' # Redacted
output_folder = 'path' # Redacted
split_video_to_frames(video_path, output_folder)

# YOLO Processing

In [14]:
# YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

def filter_images_by_object(input_folder, output_folder, object_name):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = list(Path(input_folder).glob('*.jpg'))

    for image_file in image_files:
        try:
            with Image.open(image_file) as img:
                img_array = np.array(img)
                img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
                results = model(img_array)

                df = results.pandas().xyxy[0]

                if len(df) > 0:
                    top_item = df.iloc[0]
                    top_name = top_item['name']
                    top_confidence = top_item['confidence']
                    print(f"Top item detected in {image_file.name}: {top_name} with confidence {top_confidence:.2f}")

                if any(object_name in result for result in df['name']):
                    output_path = os.path.join(output_folder, image_file.name)

                    img.save(output_path)
                    print(f"Saved image with {object_name} to {output_path}")
        except (IOError, cv2.error) as e:
            print(f"Failed to process {image_file.name}: {e}")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-8-13 Python-3.10.12 torch-2.3.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
input_folder = 'path' # Redacted
output_folder = 'path' # Redacted
object_name = 'airplane'  # Object name

filter_images_by_object(input_folder, output_folder, object_name)